# param recovery for both model

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import confusion_matrix
from joblib import Parallel, delayed
import matplotlib.tri as tri
import matplotlib.colors as mcolors
from scipy.interpolate import griddata
from scipy.interpolate import RBFInterpolator
import matplotlib.ticker as mticker
import itertools
from sklearn.metrics import r2_score
import optuna


# important directories

In [2]:
output_dir = "27_RL_agent_TDlearn_output_both_param_recovery"
os.makedirs(output_dir, exist_ok=True)


folder_path_participants = 'data_risk_added_epileptic'
folder_path_colors_numbers = '13_RL_agent_TDlearn_output/model_behavior'


df_participants = []
df_colors_numbers = []


def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_csv(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True) 
        df_participants.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".csv")
        find_matching_csv(folder_path_colors_numbers, df_colors_numbers)


In [3]:
for df in df_participants:
    df['block_type'] = None

    df.loc[df['block'] == 1, 'block_type'] = 'uniform'     # Block 1 is uni
    df.loc[df['block'] == 4, 'block_type'] = 'mix'     # Block 4 is mix

    # For blocks 2 and 3, set based on distribution
    df.loc[(df['block'] == 2) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 2) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    



for i in range(len(df_participants)):
    myCard = df_participants[i]['myCard']
    yourCard = df_participants[i]['yourCard']
    distributions = df_participants[i]['distribution']
    block_type = df_participants[i]['block_type']
    
    for df_list in [ df_colors_numbers]:
        df_list[i]['myCard'] = myCard
        df_list[i]['yourCard'] = yourCard
        df_list[i]['distribution'] = distributions
        df_list[i]['block_type'] = block_type

In [4]:
for df in df_colors_numbers:
    df['model_choices'] = df['model_choices'].replace({1: 'arrowup', 0: 'arrowdown'})


In [5]:
for df in df_colors_numbers:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 'arrowup') or (my < your and choice == 'arrowdown')):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes

In [6]:
df_participants[2]

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,is_within_IQR,risk,block_type
0,2135,uniform,835,win,4,7,14079,10.5,0,response,arrowdown,1,0,0,0.375,uniform
1,1203,uniform,926,win,9,4,1804,11,1,response,arrowup,1,0,1,0.000,uniform
2,1035,uniform,934,win,1,9,766,11.5,2,response,arrowdown,1,0,1,0.000,uniform
3,827,uniform,991,lose,2,1,1189,11,3,response,arrowdown,1,0,1,0.125,uniform
4,1306,uniform,970,win,9,3,1887,11.5,4,response,arrowup,1,0,1,0.000,uniform
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,939,uniform,773,win,4,6,802,87,24,response,arrowdown,4,0,1,0.375,mix
266,1210,high,811,win,6,8,1345,87.5,122,response,arrowdown,4,0,1,0.385,mix
267,275,high,838,win,3,5,254,88,123,response,arrowdown,4,0,1,0.071,mix
268,907,uniform,992,win,2,4,774,88.5,14,response,arrowdown,4,0,1,0.125,mix


In [7]:
df_colors_numbers[2]

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val,myCard,yourCard,distribution,block_type,outcome
0,arrowup,0,9.5,10.5,"[[[0.007151827818520263, 0.010887971741451993]...",4,7,uniform,uniform,lose
1,arrowup,1,10.0,11.0,"[[[0.007151827818520263, 0.010887971741451993]...",9,4,uniform,uniform,win
2,arrowdown,0,10.5,11.5,"[[[0.04853290651797284, 0.010887971741451993],...",1,9,uniform,uniform,win
3,arrowup,0,11.0,11.0,"[[[0.04853290651797284, 0.010887971741451993],...",2,1,uniform,uniform,win
4,arrowup,1,11.5,11.5,"[[[0.04853290651797284, 0.010887971741451993],...",9,3,uniform,uniform,win
...,...,...,...,...,...,...,...,...,...,...
265,arrowdown,0,63.0,87.0,"[[[0.3121740438868729, 0.010887971741451993], ...",4,6,uniform,mix,win
266,arrowdown,0,63.5,87.5,"[[[0.3121740438868729, 0.010887971741451993], ...",6,8,high,mix,win
267,arrowdown,0,64.0,88.0,"[[[0.3121740438868729, 0.010887971741451993], ...",3,5,high,mix,win
268,arrowdown,0,64.5,88.5,"[[[0.3121740438868729, 0.010887971741451993], ...",2,4,uniform,mix,win


In [8]:
participants = [os.path.splitext(file)[0].replace("task_data_", "")
    for file in os.listdir(folder_path_participants) if file.endswith('.csv')]

In [9]:
actions = { "arrowdown": 0, "arrowup": 1}
distributions_map = { "uniform": 0, "low": 1,  "high": 2}
card_numbers = list(range(1, 10))

# policy_table = percentage_matrix 


Q_table_init = np.random.normal(0, 0.01, (len(card_numbers), len(distributions_map), len(actions)))


Q_table = Q_table_init.copy()

#############################################################################################
# having a q-table that starts with 0! this was not a good initilization so i changed it.
# Q_table = np.zeros((len(distributions_map), len(actions)))  # 3 distributions × 2 actions
#############################################################################################

# print("policy: \n",np.shape(policy_table))
print("\n Q_table: \n",np.shape(Q_table))




 Q_table: 
 (9, 3, 2)


In [10]:
def epsilon_greedy(Q_values, epsilon):    
    probs = np.full_like(Q_values, epsilon / Q_values.shape[-1], dtype=float)  # initialize with exploration probability
    best_actions = np.argmax(Q_values, axis=-1)  # find the best action for each state
    np.put_along_axis(probs, np.expand_dims(best_actions, axis=-1), 1 - epsilon + (epsilon / Q_values.shape[-1]), axis=-1)
    return probs



def train_rescorla_wagner(df, alpha, beta, Q_init=None):
    if Q_init is None:
        Q_init = Q_table.copy()
    Q_values = Q_init.copy()
    q_value_pairs = []
    choices = []
    predicted_probs = []
    distributions = []
    card_numbers = []
    
    for _, row in df.iterrows():
        action = actions[row["model_choices"]] 
        distribution = distributions_map[row["distribution"]] 
        card_number = row["myCard"]-1 # since I'm using this as an index! I need to do -1 to make the 1 to 9 cards come to 0 to 8
        reward = 0.5 if row["outcome"] == "win" else -0.5


        probs = epsilon_greedy(Q_values, beta)
        predicted_probs.append(probs[card_number][distribution][action])
        
        prediction_error = reward - Q_values[card_number][distribution][action]
        Q_values[card_number][distribution][action] += alpha * prediction_error
        
        q_value_pairs.append(Q_values.copy())
        choices.append(action)
        distributions.append(distribution)
        card_numbers.append(card_number)
        

    return np.array(q_value_pairs), np.array(choices), np.array(predicted_probs), np.array(distributions), np.array(card_numbers)


# this is for the sake of parallel computing
def compute_log_likelihood(alpha, beta, df_all, Q_table):
    Q_init_participant = Q_table.copy()
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, alpha, beta, Q_init=Q_init_participant.copy())
    
    predicted_probs = np.clip(predicted_probs, 1e-6, 1)  # prevent log(0)
    log_likelihood = np.sum(np.log(predicted_probs))
    
    return (alpha, beta, log_likelihood)


In [11]:
num_of_samples = 100
# num_of_samples = 1000
alpha_min = 0.01
alpha_max = 1
beta_min = 0.01
beta_max  = 1
alpha_samples = np.random.uniform(alpha_min, alpha_max + np.finfo(float).eps, num_of_samples)
beta_samples = np.random.uniform(beta_min, beta_max + np.finfo(float).eps, num_of_samples)

In [12]:
BIC_models = []
AIC_models = []
best_alpha_models = []
best_beta_models = []
accuracy_models = []
precision_models = []
sensitivity_recall_models = []
specificity_models = []
f1_score_models = []
mcFadden_r2_models = []
r2_models = []

for idx, df_all in enumerate(df_colors_numbers):
    print(f"Processing participant {idx + 1} of {len(df_colors_numbers)}")
    Q_init_participant = Q_table.copy()
    
    def objective(trial):
        alpha = trial.suggest_float("alpha", alpha_min, alpha_max)
        beta  = trial.suggest_float("beta", beta_min, beta_max)

        # negative log-likelihood (Optuna minimises)
        _, _, ll = compute_log_likelihood(alpha, beta,
                                        df_all,
                                        Q_init_participant.copy())
        return -ll

    study = optuna.create_study(direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=400, n_jobs=-1)


    best_alpha = study.best_params["alpha"]
    best_beta  = study.best_params["beta"]
    best_log_likelihood = -study.best_value

    # keep this for plotting later
    results_df = study.trials_dataframe()
    results_df["alpha"] = results_df["params_alpha"]
    results_df["beta"]  = results_df["params_beta"]
    results_df["log_likelihood"] = -results_df["value"]

    # model prediction 
    
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, best_alpha, best_beta, Q_init=Q_init_participant.copy())
    
    
    predicted_choices = []
    for trial in range(len(card_numbers)):
        test_action_probs = epsilon_greedy(q_values[trial], best_beta)
        p_arrowup = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowup"]]
        p_arrow_down = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowdown"]]
        # choosing 1 or 0 based on the softmax probabilities:
        predicted_choice = int(p_arrowup >= 0.5)  # 1 if up-prob ≥ 0.5 else 0
        predicted_choices.append(predicted_choice)

    # finding out model total reward based on the model's predicted choices
    total_reward = [] 
    for i in range(len(predicted_choices)):
        if len(total_reward)> 0:
            last_reward = total_reward[-1]  #  the last reward value
        else:
            last_reward = 10 # initial reward is $10
        
        if ((df_all.loc[i, 'myCard'] > df_all.loc[i, 'yourCard'] and predicted_choices[i] == 1) or
            (df_all.loc[i, 'myCard'] < df_all.loc[i, 'yourCard'] and predicted_choices[i] == 0)):
            total_reward.append(last_reward + 0.5)
        else:
            total_reward.append(last_reward - 0.5)

    
   
       # confusion matrix:
    conf_matrix = confusion_matrix(choices, predicted_choices)
    TN, FP, FN, TP = conf_matrix.ravel()  # unpacking the confusion matrix
    # acc
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    # precision: From the ones that we’ve announced them as up/down, which ones are really up/down?
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    # recall or sensitivity : true positive rate
    sensitivity_recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    # specificity : true negative rate
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    # f1 Score
    f1_score = 2 * (precision * sensitivity_recall) / (precision + sensitivity_recall) if (precision + sensitivity_recall) != 0 else 0

    
    # bayes information criterion:
    n_trials = len(df_all)
    k = 2  # number of free parameters: alpha and beta
    BIC = k * np.log(n_trials) - 2 * best_log_likelihood # this is BIC formula based on the log lkelihode I found before

    

    # Akaike  information criterion(AIC):
    AIC = 2 * k - 2 * best_log_likelihood 


    # mcFadden r-squared:
    p_null = np.mean(choices)  # probability of choosing "1" in the dataset
    log_likelihood_null = np.sum(choices * np.log(p_null) + (1 - choices) * np.log(1 - p_null))
    mcFadden_r2 = 1 - (best_log_likelihood / log_likelihood_null)

    # r-squared
    r2 = r2_score(choices, predicted_choices)
    print(best_alpha)
    print(best_beta)
    
    # saving models evaluation variables:
    best_alpha_models.append(best_alpha)
    best_beta_models.append(best_beta)
    BIC_models.append(BIC)
    AIC_models.append(AIC)
    accuracy_models.append(accuracy)
    precision_models.append(precision)
    sensitivity_recall_models.append(sensitivity_recall)
    specificity_models.append(specificity)
    f1_score_models.append(f1_score)
    mcFadden_r2_models.append(mcFadden_r2)
    r2_models.append(r2)





[I 2025-06-18 15:34:05,023] A new study created in memory with name: no-name-02f9ebfc-981d-47f2-a5e7-239013015012


Processing participant 1 of 7


[I 2025-06-18 15:34:05,501] Trial 0 finished with value: 203.03507832435977 and parameters: {'alpha': 0.9540741244951494, 'beta': 0.22737575305500368}. Best is trial 0 with value: 203.03507832435977.
[I 2025-06-18 15:34:05,623] Trial 4 finished with value: 166.95922305503262 and parameters: {'alpha': 0.3421458640715747, 'beta': 0.6957826694617425}. Best is trial 4 with value: 166.95922305503262.
[I 2025-06-18 15:34:05,653] Trial 3 finished with value: 167.2027327133847 and parameters: {'alpha': 0.7078114194356206, 'beta': 0.6779361002327008}. Best is trial 4 with value: 166.95922305503262.
[I 2025-06-18 15:34:05,672] Trial 2 finished with value: 191.4237821421312 and parameters: {'alpha': 0.5734464514080082, 'beta': 0.27982676335530976}. Best is trial 4 with value: 166.95922305503262.
[I 2025-06-18 15:34:05,699] Trial 1 finished with value: 171.50533931129542 and parameters: {'alpha': 0.31796877093033293, 'beta': 0.8127774829908128}. Best is trial 4 with value: 166.95922305503262.
[I 2

0.11163454880052036
0.5854282568869038
Processing participant 2 of 7


[I 2025-06-18 15:34:46,066] Trial 4 finished with value: 172.9260486923471 and parameters: {'alpha': 0.7040331594190757, 'beta': 0.506467336069113}. Best is trial 4 with value: 172.9260486923471.
[I 2025-06-18 15:34:46,073] Trial 5 finished with value: 197.98136126026006 and parameters: {'alpha': 0.3701385070144313, 'beta': 0.27437626779060037}. Best is trial 4 with value: 172.9260486923471.
[I 2025-06-18 15:34:46,081] Trial 2 finished with value: 180.13428343992422 and parameters: {'alpha': 0.751959767209861, 'beta': 0.9175081094489098}. Best is trial 4 with value: 172.9260486923471.
[I 2025-06-18 15:34:46,111] Trial 6 finished with value: 169.43065748751752 and parameters: {'alpha': 0.391321192199543, 'beta': 0.6933395289615899}. Best is trial 6 with value: 169.43065748751752.
[I 2025-06-18 15:34:46,120] Trial 3 finished with value: 172.50440527460256 and parameters: {'alpha': 0.7525894444091186, 'beta': 0.7832965739313014}. Best is trial 6 with value: 169.43065748751752.
[I 2025-06-

...

0.12874917238173234
0.6424877430316458
Processing participant 6 of 7


[I 2025-06-18 15:37:18,683] Trial 0 finished with value: 176.01921551672626 and parameters: {'alpha': 0.4786360692293391, 'beta': 0.6271115378955243}. Best is trial 0 with value: 176.01921551672626.
[I 2025-06-18 15:37:18,723] Trial 1 finished with value: 324.58191126520086 and parameters: {'alpha': 0.7536037679881098, 'beta': 0.07010393884950678}. Best is trial 0 with value: 176.01921551672626.
[I 2025-06-18 15:37:18,786] Trial 2 finished with value: 184.32917613655533 and parameters: {'alpha': 0.5091613068640474, 'beta': 0.9623694533178268}. Best is trial 0 with value: 176.01921551672626.
[I 2025-06-18 15:37:18,889] Trial 8 finished with value: 176.87902917468216 and parameters: {'alpha': 0.1246326319464685, 'beta': 0.7284960076996273}. Best is trial 0 with value: 176.01921551672626.
[I 2025-06-18 15:37:19,062] Trial 4 finished with value: 301.02479885848027 and parameters: {'alpha': 0.513696147832461, 'beta': 0.09171570285236019}. Best is trial 0 with value: 176.01921551672626.
[I 2

0.42213734576466444
0.6953595945687586
Processing participant 7 of 7


[I 2025-06-18 15:37:55,696] Trial 2 finished with value: 206.4871103703333 and parameters: {'alpha': 0.654884414284309, 'beta': 0.2612592730586461}. Best is trial 0 with value: 170.34789825080205.
[I 2025-06-18 15:37:55,938] Trial 5 finished with value: 275.5894355045711 and parameters: {'alpha': 0.8132690728908787, 'beta': 0.10039702029903803}. Best is trial 0 with value: 170.34789825080205.
[I 2025-06-18 15:37:56,030] Trial 6 finished with value: 212.0895560891561 and parameters: {'alpha': 0.09694573951036052, 'beta': 0.2317008452327616}. Best is trial 0 with value: 170.34789825080205.
[I 2025-06-18 15:37:56,084] Trial 8 finished with value: 439.01087883608614 and parameters: {'alpha': 0.3506580095723883, 'beta': 0.013825138484098227}. Best is trial 0 with value: 170.34789825080205.
[I 2025-06-18 15:37:56,114] Trial 4 finished with value: 172.14052398831257 and parameters: {'alpha': 0.6740811619391391, 'beta': 0.5808142480968868}. Best is trial 0 with value: 170.34789825080205.
[I 20

0.10285759055798023
0.6318667787586741


# now saving the model evaluation values

In [13]:
df_models_evaluation = pd.DataFrame({
    "participants": participants,
    "best_alpha": best_alpha_models,
    "best_beta": best_beta_models,
    "BIC": BIC_models,
    "AIC": AIC_models,
    "accuracy": accuracy_models,
    "precision": precision_models,
    "sensitivity_recall": sensitivity_recall_models,
    "specificity": specificity_models,
    "f1_score": f1_score_models,
    "mcFadden_r2": mcFadden_r2_models,
    "r2": r2_models
})

file_path = os.path.join(output_dir, "models_evaluation_greedy.csv")
df_models_evaluation.to_csv(file_path, index=False)